Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [12]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [13]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [14]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [15]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune <b>L2 regularization </b> for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [21]:
batch_size = 128
hidden_nodes = 1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  input_weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_nodes]))
  input_biases = tf.Variable(tf.zeros([hidden_nodes]))

  output_weights = tf.Variable(
    tf.truncated_normal([hidden_nodes, num_labels]))
  output_biases = tf.Variable(tf.zeros([num_labels]))    

  tf_beta_reg = tf.Variable(0.1)

  # Training computation.
  input = tf.matmul(tf_train_dataset, input_weights) + input_biases

  hidden_layer = tf.nn.relu(input)

  logits = tf.matmul(hidden_layer, output_weights) + output_biases

  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) \
    + 0.0005*(tf.nn.l2_loss(input_weights) + tf.nn.l2_loss(output_weights))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, input_weights) + input_biases),output_weights) + output_biases
  )
  test_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, input_weights) + input_biases),output_weights) + output_biases
  )

In [23]:
num_steps = 20001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 397.499817
Minibatch accuracy: 15.6%
Validation accuracy: 23.4%
Minibatch loss at step 500: 134.093567
Minibatch accuracy: 84.4%
Validation accuracy: 79.7%
Minibatch loss at step 1000: 98.803970
Minibatch accuracy: 80.5%
Validation accuracy: 80.5%
Minibatch loss at step 1500: 73.437386
Minibatch accuracy: 87.5%
Validation accuracy: 82.4%
Minibatch loss at step 2000: 56.239384
Minibatch accuracy: 90.6%
Validation accuracy: 82.0%
Minibatch loss at step 2500: 43.489956
Minibatch accuracy: 89.1%
Validation accuracy: 83.7%
Minibatch loss at step 3000: 34.089489
Minibatch accuracy: 82.8%
Validation accuracy: 84.9%
Minibatch loss at step 3500: 26.412790
Minibatch accuracy: 85.9%
Validation accuracy: 85.3%
Minibatch loss at step 4000: 20.715872
Minibatch accuracy: 88.3%
Validation accuracy: 85.8%
Minibatch loss at step 4500: 16.165342
Minibatch accuracy: 90.6%
Validation accuracy: 86.2%
Minibatch loss at step 5000: 12.720320
Minibatch accuracy: 88.3%
Valid

---
Problem 2
---------
Let's demonstrate an <b>extreme case of overfitting</b>. Restrict your training data to just a few batches. What happens?

---

In [24]:
batch_size = 128
hidden_nodes = 1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  input_weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_nodes]))
  input_biases = tf.Variable(tf.zeros([hidden_nodes]))

  output_weights = tf.Variable(
    tf.truncated_normal([hidden_nodes, num_labels]))
  output_biases = tf.Variable(tf.zeros([num_labels]))    

  tf_beta_reg = tf.Variable(0.1)

  # Training computation.
  input = tf.matmul(tf_train_dataset, input_weights) + input_biases

  hidden_layer = tf.nn.relu(input)

  logits = tf.matmul(hidden_layer, output_weights) + output_biases

  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) \
    + 0.0005*(tf.nn.l2_loss(input_weights) + tf.nn.l2_loss(output_weights))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, input_weights) + input_biases),output_weights) + output_biases
  )
  test_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, input_weights) + input_biases),output_weights) + output_biases
  )

In [28]:
num_steps = 6001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    #offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    offset = (step * batch_size) % (batch_size*10 - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 543.198120
Minibatch accuracy: 13.3%
Validation accuracy: 33.3%
Minibatch loss at step 500: 122.405159
Minibatch accuracy: 100.0%
Validation accuracy: 77.2%
Minibatch loss at step 1000: 95.326157
Minibatch accuracy: 100.0%
Validation accuracy: 77.2%
Minibatch loss at step 1500: 74.237793
Minibatch accuracy: 100.0%
Validation accuracy: 77.2%
Minibatch loss at step 2000: 57.814587
Minibatch accuracy: 100.0%
Validation accuracy: 77.2%
Minibatch loss at step 2500: 45.024708
Minibatch accuracy: 100.0%
Validation accuracy: 77.3%
Minibatch loss at step 3000: 35.064262
Minibatch accuracy: 100.0%
Validation accuracy: 77.3%
Minibatch loss at step 3500: 27.307390
Minibatch accuracy: 100.0%
Validation accuracy: 77.3%
Minibatch loss at step 4000: 21.266426
Minibatch accuracy: 100.0%
Validation accuracy: 77.4%
Minibatch loss at step 4500: 16.562237
Minibatch accuracy: 100.0%
Validation accuracy: 77.3%
Minibatch loss at step 5000: 12.898880
Minibatch accuracy: 10

---
Problem 3
---------
<b>Introduce Dropout on the hidden layer</b> of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [29]:
batch_size = 128
hidden_nodes = 1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  input_weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_nodes]))
  input_biases = tf.Variable(tf.zeros([hidden_nodes]))

  output_weights = tf.Variable(
    tf.truncated_normal([hidden_nodes, num_labels]))
  output_biases = tf.Variable(tf.zeros([num_labels]))    

  tf_beta_reg = tf.Variable(0.1)

  # Training computation.
  input = tf.matmul(tf_train_dataset, input_weights) + input_biases

  hidden_layer = tf.nn.dropout(tf.nn.relu(input),0.6) ## add dropout

  logits = tf.matmul(hidden_layer, output_weights) + output_biases

  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) \
    + 0.0005*(tf.nn.l2_loss(input_weights) + tf.nn.l2_loss(output_weights))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, input_weights) + input_biases),output_weights) + output_biases
  )
  test_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, input_weights) + input_biases),output_weights) + output_biases
  )

In [31]:
num_steps = 20001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    #offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    offset = (step * batch_size) % (batch_size*10 - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 561.331360
Minibatch accuracy: 11.7%
Validation accuracy: 22.3%
Minibatch loss at step 500: 123.303497
Minibatch accuracy: 99.2%
Validation accuracy: 78.6%
Minibatch loss at step 1000: 95.695740
Minibatch accuracy: 100.0%
Validation accuracy: 79.5%
Minibatch loss at step 1500: 74.552902
Minibatch accuracy: 100.0%
Validation accuracy: 79.7%
Minibatch loss at step 2000: 58.071941
Minibatch accuracy: 100.0%
Validation accuracy: 79.6%
Minibatch loss at step 2500: 45.233986
Minibatch accuracy: 100.0%
Validation accuracy: 79.3%
Minibatch loss at step 3000: 35.342239
Minibatch accuracy: 98.4%
Validation accuracy: 79.9%
Minibatch loss at step 3500: 27.460169
Minibatch accuracy: 99.2%
Validation accuracy: 80.0%
Minibatch loss at step 4000: 21.372314
Minibatch accuracy: 100.0%
Validation accuracy: 79.8%
Minibatch loss at step 4500: 16.645037
Minibatch accuracy: 100.0%
Validation accuracy: 79.7%
Minibatch loss at step 5000: 12.963511
Minibatch accuracy: 100.0

---
Problem 4
---------

Try to get the best performance you can using a <b>multi-layer model</b>! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use <b>learning rate decay</b>:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [42]:
batch_size = 128
hidden_nodes = 1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  input_weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_nodes]))
  input_biases = tf.Variable(tf.zeros([hidden_nodes]))

  output_weights = tf.Variable(
    tf.truncated_normal([hidden_nodes, num_labels]))
  output_biases = tf.Variable(tf.zeros([num_labels]))    

  tf_beta_reg = tf.Variable(0.1)

  # Training computation.
  input = tf.matmul(tf_train_dataset, input_weights) + input_biases

  hidden_layer = tf.nn.dropout(tf.nn.relu(input),0.6) ## add dropout

  logits = tf.matmul(hidden_layer, output_weights) + output_biases

  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) \
    + 0.0005*(tf.nn.l2_loss(input_weights) + tf.nn.l2_loss(output_weights))
  
  # Optimizer with decayed learning rate
  global_step = tf.Variable(0)  # count the number of steps taken.
  learning_rate = tf.train.exponential_decay(0.5, global_step,10000,0.8, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss,global_step = global_step)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, input_weights) + input_biases),output_weights) + output_biases
  )
  test_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, input_weights) + input_biases),output_weights) + output_biases
  )

In [43]:
num_steps =30001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    #offset = (step * batch_size) % (batch_size*10 - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 1000 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 530.265198
Minibatch accuracy: 10.2%
Validation accuracy: 31.3%
Minibatch loss at step 1000: 101.340042
Minibatch accuracy: 72.7%
Validation accuracy: 80.3%
Minibatch loss at step 2000: 56.924328
Minibatch accuracy: 83.6%
Validation accuracy: 82.3%
Minibatch loss at step 3000: 33.884624
Minibatch accuracy: 83.6%
Validation accuracy: 83.3%
Minibatch loss at step 4000: 20.711287
Minibatch accuracy: 85.9%
Validation accuracy: 85.0%
Minibatch loss at step 5000: 12.569520
Minibatch accuracy: 88.3%
Validation accuracy: 86.0%
Minibatch loss at step 6000: 7.983128
Minibatch accuracy: 78.9%
Validation accuracy: 86.6%
Minibatch loss at step 7000: 5.021873
Minibatch accuracy: 83.6%
Validation accuracy: 87.2%
Minibatch loss at step 8000: 3.510082
Minibatch accuracy: 76.6%
Validation accuracy: 87.3%
Minibatch loss at step 9000: 2.222075
Minibatch accuracy: 86.7%
Validation accuracy: 88.1%
Minibatch loss at step 10000: 1.530872
Minibatch accuracy: 83.6%
Validati